In [26]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

from datetime import datetime
from keras.models import Sequential
from keras.layers import Dense,RepeatVector
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D,Conv2D
from tensorflow.keras.layers import Conv2D
from keras.layers.convolutional import MaxPooling1D,MaxPooling2D
import tensorflow
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.callbacks import EarlyStopping
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import  SimpleRNN
from statsmodels.tools.eval_measures import rmse 

from scipy.stats import wilcoxon
from scipy.stats import stats

In [2]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (0, 1))

In [3]:
def x_y_split(df,ratio):
    interval=int(len(df)*ratio)
    train = df[:interval]
    test = df[interval:]
    train.dropna(inplace=True)
    test.dropna(inplace=True)
    return train,test

In [4]:
def convert2matrix(data_arr, look_back):
 X, Y =[], []
 for i in range(len(data_arr)-look_back):
  d=i+look_back  
  X.append(data_arr[i:d,0])
  Y.append(data_arr[d,0])
 return np.array(X), np.array(Y)

In [5]:
def model_dnn(look_back):
    model=Sequential()
    model.add(Dense(units=32, input_dim=look_back, activation='relu'))
    model.add(Dense(8, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error',  optimizer='adam',metrics = ['mse', 'mae'])
    return model

In [6]:
def model_cnn(xtrain):
    model = Sequential()
    model.add(Conv1D(filters=128, kernel_size=2, activation='relu', input_shape=(xtrain.shape[1], 1)))
    model.add(Conv1D(filters=128, kernel_size=2, activation='relu'))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [7]:
def model_lstm(xtrain):
    model = Sequential()
    model.add(LSTM(100, return_sequences=True, input_shape=(xtrain.shape[1], xtrain.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(50))
    model.add(Dropout(0.1))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [8]:
def model_cnn_lstm(xtrain):
    model_cnn_lstm = Sequential()
    model_cnn_lstm.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, xtrain.shape[2], xtrain.shape[3])))
    model_cnn_lstm.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model_cnn_lstm.add(TimeDistributed(Flatten()))
    model_cnn_lstm.add(LSTM(50, activation='relu'))
    #model_cnn_lstm.add(LSTM(10, stateful=True))
    model_cnn_lstm.add(Dense(1))
    model_cnn_lstm.compile(loss='mse', optimizer='adam')
    return model_cnn_lstm

In [9]:
def model_rnn(look_back):
  model=Sequential()
  model.add(SimpleRNN(units=32, input_shape=(look_back,1), activation="relu"))
  model.add(Dense(8, activation='relu'))
  model.add(Dense(1))
  model.compile(loss='mean_squared_error',  optimizer='adam',metrics = ['mse', 'mae'])
  return model
    

In [10]:
def model_ann(look_back):
    model = Sequential()
    model.add(Dense(12, input_dim=look_back, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [11]:
def preprocessing_walmart(df,scaler):
    df=df.drop(columns=['Store','Dept','IsHoliday'])
    scaler=scaler.fit(df)
    df['Weekly_Sales']=scaler.transform(df)
    return df
    
    

In [12]:
def preprocess_grocery(df,scaler):
    df.drop(['id','store_nbr','item_nbr','onpromotion'], axis=1,inplace=True)
    scaler=scaler.fit(df)
    df['unit_sales']=scaler.transform(df)
    return df

In [13]:
def reshape3d(data):
    return data.reshape(data.shape[0],data.shape[1],1)
    

In [14]:
def reshape4d(data,x,y):
    return data.reshape((data.shape[0],x, y,1))

In [15]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [16]:
def preprocess_weather(df,scaler):
    df=df[['temperatureMax']]
    scaler=scaler.fit(df)
    df['temperatureMax']=scaler.transform(df)
    return df

In [17]:
df=pd.read_csv("Daily_Weather_Splitted_Data/Albania.csv",parse_dates=['time'],index_col='time')

In [50]:
df=preprocess_weather(df,scaler)

In [18]:
import os
import logging, sys
logging.disable(sys.maxsize)
from math import sqrt
paths_to_folders = ['C:/Users/RAHAT/Downloads/Untitled Folder/Sales_f/data/Practice/walmart']
x=0
scaler = MinMaxScaler(feature_range = (0, 1))
warnings.filterwarnings("ignore")
# error = pd.DataFrame(columns=['File_name', 'ARIMA_RMSE_ERROR','SES_RMSE_ERROR','SARIMA_RMSE_ERROR','PROPHET_RMSE_ERROR',
#                               'HOLT_WINTER_RMSE_ERROR','ARIMA_MAPE_ERROR','SES_MAPE_ERROR','SARIMA_MAPE_ERROR','PROPHET_MAPE_ERROR',
#                               'HOLT_WINTER_MAPE_ERROR'])
error = pd.DataFrame(columns=['File_name','ANN_MAPE','CNN_MAPE', 'DNN_MAPE','LSTM_MAPE','RNN_MAPE','CNN+LSTM_MAPE','ANN_RMSE','CNN_RMSE','DNN_RMSE','LSTM_RMSE','RNN_RMSE','CNN+LSTM_RMSE'])
for folder in paths_to_folders:
   for csv_file in os.listdir(folder):
#      df=pd.read_csv("C:/Users/RAHAT/Downloads/Untitled Folder/Sales_f/data/Practice/weather"+csv_file,parse_dates=['Date'],index_col='Date')
     df=pd.read_csv("C:/Users/RAHAT/Downloads/Untitled Folder/Sales_f/data/Practice/walmart/"+csv_file,parse_dates=['Date'],index_col='Date')
     #print(len(df.index);
#      if len(df.index)>=140:
     if len(df.index)>=100:
         test_predictions=pd.DataFrame()
         df=preprocessing_walmart(df,scaler)
         train,test=x_y_split(df,0.8)
         timestep=4
         xtrain,ytrain=convert2matrix(train.values,timestep)
         xtest,ytest=convert2matrix(test.values,timestep)
         test_predictions['Weekly_sales']=np.array(ytest)
        
         #DNN MODEL
         model = model_dnn(timestep)
         model.fit(xtrain,ytrain, epochs=60, batch_size=32, verbose=1,callbacks=[EarlyStopping(monitor='val_loss', patience=10)],shuffle=False)
         test_predict = model.predict(xtest)
         test_predictions['DNN']=np.array(test_predict)
            
        
         #RNN Model  
         model = model_rnn(timestep)
         model.fit(reshape3d(xtrain),ytrain,epochs=100, batch_size=30, verbose=1,callbacks=[EarlyStopping(monitor='val_loss', patience=10)],shuffle=False)
         test_predict = model.predict(reshape3d(xtest))
         test_predictions['RNN']=np.array(test_predict)
        
            
         #CNN MODEL
         model=model_cnn(reshape3d(xtrain))
         model.fit(reshape3d(xtrain), ytrain, epochs=60, verbose=1)
         cnn_preds = model.predict(reshape3d(xtest))
         test_predictions['CNN']=np.array(cnn_preds)
         
         #ANN_MODEL
         model=model_ann(timestep)
         early_stop = EarlyStopping(monitor='loss', patience=2, verbose=1)
         model.fit(xtrain, ytrain, epochs=100, batch_size=32, verbose=2, callbacks=[early_stop],shuffle=False)
         ann_preds = model.predict(xtest)
         test_predictions['ANN']=np.array(ann_preds)  

            
         #LSTM MODEL
         model=model_lstm(reshape3d(xtrain))
         model.fit(reshape3d(xtrain), ytrain, epochs=60, batch_size=32,  verbose=1, shuffle=False)
         lstm_preds = model.predict(reshape3d(xtest))
         test_predictions['LSTM']=np.array(lstm_preds)
        
        
         #CNN+LSTM
         model=model_cnn_lstm(reshape4d(xtrain,2,2))
         model.fit(reshape4d(xtrain,2,2), ytrain, epochs=60, verbose=2) 
         predictions = model.predict(reshape4d(xtest,2,2))
         test_predictions['CNN+LSTM']=np.array(predictions)
         
        
        
        
         error.loc[x, ['File_name']]=csv_file
         error.loc[x, ['DNN_MAPE']]=mean_absolute_percentage_error(test_predictions['Weekly_sales'],test_predictions['DNN'])
         error.loc[x, ['CNN_MAPE']]=mean_absolute_percentage_error(test_predictions['Weekly_sales'],test_predictions['CNN'])
         error.loc[x, ['ANN_MAPE']]=mean_absolute_percentage_error(test_predictions['Weekly_sales'],test_predictions['ANN'])
         error.loc[x, ['RNN_MAPE']]=mean_absolute_percentage_error(test_predictions['Weekly_sales'],test_predictions['RNN'])
         error.loc[x, ['LSTM_MAPE']]=mean_absolute_percentage_error(test_predictions['Weekly_sales'],test_predictions['LSTM'])
         error.loc[x, ['CNN+LSTM_MAPE']]=mean_absolute_percentage_error(test_predictions['Weekly_sales'],test_predictions['CNN+LSTM'])
        
         error.loc[x, ['DNN_RMSE']]=rmse(test_predictions['Weekly_sales'],test_predictions['DNN'])
         error.loc[x, ['CNN_RMSE']]=rmse(test_predictions['Weekly_sales'],test_predictions['CNN'])
         error.loc[x, ['ANN_RMSE']]=rmse(test_predictions['Weekly_sales'],test_predictions['ANN'])
         error.loc[x, ['LSTM_RMSE']]=rmse(test_predictions['Weekly_sales'],test_predictions['LSTM'])
         error.loc[x, ['RNN_RMSE']]=rmse(test_predictions['Weekly_sales'],test_predictions['RNN'])
         error.loc[x, ['CNN+LSTM_RMSE']]=rmse(test_predictions['Weekly_sales'],test_predictions['CNN+LSTM'])
       

        
        
         x=x+1
         print(csv_file)
         print(x)
         #print(rmse(test["unit_sales"], predictions))
         #print(mean_squared_error(test["unit_sales"], predictions))

Epoch 1/60
110/110 [==============================] - 0s 2ms/step - loss: 0.1174 - mse: 0.1174 - mae: 0.2784
Epoch 2/60
110/110 [==============================] - 0s 73us/step - loss: 0.0896 - mse: 0.0896 - mae: 0.2318
Epoch 3/60
110/110 [==============================] - 0s 105us/step - loss: 0.0690 - mse: 0.0690 - mae: 0.1959
Epoch 4/60
110/110 [==============================] - 0s 133us/step - loss: 0.0549 - mse: 0.0549 - mae: 0.1709
Epoch 5/60
110/110 [==============================] - 0s 251us/step - loss: 0.0453 - mse: 0.0453 - mae: 0.1544
Epoch 6/60
110/110 [==============================] - 0s 208us/step - loss: 0.0388 - mse: 0.0388 - mae: 0.1427
Epoch 7/60
110/110 [==============================] - 0s 95us/step - loss: 0.0346 - mse: 0.0346 - mae: 0.1355
Epoch 8/60
110/110 [==============================] - 0s 268us/step - loss: 0.0321 - mse: 0.0321 - mae: 0.1316
Epoch 9/60
110/110 [==============================] - 0s 116us/step - loss: 0.0307 - mse: 0.0307 - mae: 0.1297
Epoch

110/110 [==============================] - 0s 180us/step - loss: 0.0256 - mse: 0.0256 - mae: 0.1268
Epoch 14/100
110/110 [==============================] - 0s 179us/step - loss: 0.0253 - mse: 0.0253 - mae: 0.1257
Epoch 15/100
110/110 [==============================] - 0s 92us/step - loss: 0.0249 - mse: 0.0249 - mae: 0.1251
Epoch 16/100
110/110 [==============================] - 0s 180us/step - loss: 0.0247 - mse: 0.0247 - mae: 0.1250
Epoch 17/100
110/110 [==============================] - 0s 88us/step - loss: 0.0245 - mse: 0.0245 - mae: 0.1251
Epoch 18/100
110/110 [==============================] - 0s 92us/step - loss: 0.0244 - mse: 0.0244 - mae: 0.1251
Epoch 19/100
110/110 [==============================] - 0s 90us/step - loss: 0.0243 - mse: 0.0243 - mae: 0.1247
Epoch 20/100
110/110 [==============================] - 0s 94us/step - loss: 0.0242 - mse: 0.0242 - mae: 0.1241
Epoch 21/100
110/110 [==============================] - 0s 157us/step - loss: 0.0239 - mse: 0.0239 - mae: 0.1233
E

110/110 [==============================] - 0s 249us/step - loss: 0.0183 - mse: 0.0183 - mae: 0.1048
Epoch 86/100
110/110 [==============================] - 0s 92us/step - loss: 0.0183 - mse: 0.0183 - mae: 0.1048
Epoch 87/100
110/110 [==============================] - 0s 138us/step - loss: 0.0183 - mse: 0.0183 - mae: 0.1046
Epoch 88/100
110/110 [==============================] - 0s 182us/step - loss: 0.0182 - mse: 0.0182 - mae: 0.1045
Epoch 89/100
110/110 [==============================] - ETA: 0s - loss: 0.0092 - mse: 0.0092 - mae: 0.077 - 0s 90us/step - loss: 0.0182 - mse: 0.0182 - mae: 0.1045
Epoch 90/100
110/110 [==============================] - 0s 174us/step - loss: 0.0181 - mse: 0.0181 - mae: 0.1044
Epoch 91/100
110/110 [==============================] - 0s 158us/step - loss: 0.0181 - mse: 0.0181 - mae: 0.1042
Epoch 92/100
110/110 [==============================] - 0s 205us/step - loss: 0.0180 - mse: 0.0180 - mae: 0.1040
Epoch 93/100
110/110 [==============================] - 0s 

Epoch 39/100
 - 0s - loss: 0.0261
Epoch 40/100
 - 0s - loss: 0.0259
Epoch 41/100
 - 0s - loss: 0.0258
Epoch 42/100
 - 0s - loss: 0.0256
Epoch 43/100
 - 0s - loss: 0.0255
Epoch 44/100
 - 0s - loss: 0.0254
Epoch 45/100
 - 0s - loss: 0.0253
Epoch 46/100
 - 0s - loss: 0.0252
Epoch 47/100
 - 0s - loss: 0.0251
Epoch 48/100
 - 0s - loss: 0.0250
Epoch 49/100
 - 0s - loss: 0.0249
Epoch 50/100
 - 0s - loss: 0.0248
Epoch 51/100
 - 0s - loss: 0.0247
Epoch 52/100
 - 0s - loss: 0.0246
Epoch 53/100
 - 0s - loss: 0.0245
Epoch 54/100
 - 0s - loss: 0.0244
Epoch 55/100
 - 0s - loss: 0.0243
Epoch 56/100
 - 0s - loss: 0.0242
Epoch 57/100
 - 0s - loss: 0.0241
Epoch 58/100
 - 0s - loss: 0.0241
Epoch 59/100
 - 0s - loss: 0.0240
Epoch 60/100
 - 0s - loss: 0.0239
Epoch 61/100
 - 0s - loss: 0.0238
Epoch 62/100
 - 0s - loss: 0.0238
Epoch 63/100
 - 0s - loss: 0.0237
Epoch 64/100
 - 0s - loss: 0.0236
Epoch 65/100
 - 0s - loss: 0.0236
Epoch 66/100
 - 0s - loss: 0.0235
Epoch 67/100
 - 0s - loss: 0.0234
Epoch 68/100
 

Epoch 34/60
 - 0s - loss: 0.0258
Epoch 35/60
 - 0s - loss: 0.0258
Epoch 36/60
 - 0s - loss: 0.0257
Epoch 37/60
 - 0s - loss: 0.0256
Epoch 38/60
 - 0s - loss: 0.0257
Epoch 39/60
 - 0s - loss: 0.0257
Epoch 40/60
 - 0s - loss: 0.0256
Epoch 41/60
 - 0s - loss: 0.0256
Epoch 42/60
 - 0s - loss: 0.0256
Epoch 43/60
 - 0s - loss: 0.0257
Epoch 44/60
 - 0s - loss: 0.0257
Epoch 45/60
 - 0s - loss: 0.0256
Epoch 46/60
 - 0s - loss: 0.0256
Epoch 47/60
 - 0s - loss: 0.0256
Epoch 48/60
 - 0s - loss: 0.0257
Epoch 49/60
 - 0s - loss: 0.0257
Epoch 50/60
 - 0s - loss: 0.0257
Epoch 51/60
 - 0s - loss: 0.0256
Epoch 52/60
 - 0s - loss: 0.0255
Epoch 53/60
 - 0s - loss: 0.0257
Epoch 54/60
 - 0s - loss: 0.0257
Epoch 55/60
 - 0s - loss: 0.0256
Epoch 56/60
 - 0s - loss: 0.0256
Epoch 57/60
 - 0s - loss: 0.0255
Epoch 58/60
 - 0s - loss: 0.0255
Epoch 59/60
 - 0s - loss: 0.0255
Epoch 60/60
 - 0s - loss: 0.0257
S_3__D_2.csv
1


In [19]:
error

,File_name,ANN_MAPE,CNN_MAPE,DNN_MAPE,LSTM_MAPE,RNN_MAPE,CNN+LSTM_MAPE,ANN_RMSE,CNN_RMSE,DNN_RMSE,LSTM_RMSE,RNN_RMSE,CNN+LSTM_RMSE
0,S_3__D_2.csv,13.3303,15.0323,13.0345,13.8656,13.808,13.6639,0.120606,0.124062,0.118786,0.123671,0.115877,0.131179


In [30]:
for column in test_predictions.columns:
    if column!='ANN':
        print('-----------',column,'----------')
        stat, p,z = wilcoxon(test_predictions['ANN'],test_predictions[column])
        print('Statistics=%.3f, p=%.3f' % (stat, p))
        print(p)
        print(z)

        # interpret
        alpha = 0.05
        if p > alpha:
            print('Same distribution (fail to reject H0)')
        else:
            print('Different distribution (reject H0)')
        
    

----------- Weekly_sales ----------
Statistics=87.000, p=0.042
0.04220715353440861
-2.031471837868864
Different distribution (reject H0)
----------- DNN ----------
Statistics=124.000, p=0.300
0.3002412845490545
-1.0359161027543213
Same distribution (fail to reject H0)
----------- RNN ----------
Statistics=69.000, p=0.012
0.011876381887516064
-2.5157962495462085
Different distribution (reject H0)
----------- CNN ----------
Statistics=130.000, p=0.382
0.3818598292804778
-0.8744746321952063
Same distribution (fail to reject H0)
----------- LSTM ----------
Statistics=158.000, p=0.904
0.9036267967285596
-0.12108110291933624
Same distribution (fail to reject H0)
----------- CNN+LSTM ----------
Statistics=125.000, p=0.313
0.3129702263334466
-1.0090091909944687
Same distribution (fail to reject H0)


In [27]:
from numpy import (asarray, compress) # requiered for local modification
from scipy.stats import (find_repeats, distributions)
from math import sqrt
def wilcoxon(x, y=None, zero_method="wilcox", correction=False):
    """
    Calculate the Wilcoxon signed-rank test.

    The Wilcoxon signed-rank test tests the null hypothesis that two
    related paired samples come from the same distribution. In particular,
    it tests whether the distribution of the differences x - y is symmetric
    about zero. It is a non-parametric version of the paired T-test.

    Parameters
    ----------
    x : array_like
        The first set of measurements.
    y : array_like, optional
        The second set of measurements.  If `y` is not given, then the `x`
        array is considered to be the differences between the two sets of
        measurements.
    zero_method : string, {"pratt", "wilcox", "zsplit"}, optional
        "pratt":
            Pratt treatment: includes zero-differences in the ranking process
            (more conservative)
        "wilcox":
            Wilcox treatment: discards all zero-differences
        "zsplit":
            Zero rank split: just like Pratt, but splitting the zero rank
            between positive and negative ones
    correction : bool, optional
        If True, apply continuity correction by adjusting the Wilcoxon rank
        statistic by 0.5 towards the mean value when computing the
        z-statistic.  Default is False.

    Returns
    -------
    statistic : float
        The sum of the ranks of the differences above or below zero, whichever
        is smaller.
    pvalue : float
        The two-sided p-value for the test.

    Notes
    -----
    Because the normal approximation is used for the calculations, the
    samples used should be large.  A typical rule is to require that
    n > 20.

    References
    ----------
    .. [1] https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test

    """


    if zero_method not in ["wilcox", "pratt", "zsplit"]:
        raise ValueError("Zero method should be either 'wilcox' "
                         "or 'pratt' or 'zsplit'")


    if y is None:
        d = asarray(x)
    else:
        x, y = map(asarray, (x, y))
        if len(x) != len(y):
            raise ValueError('Unequal N in wilcoxon.  Aborting.')
        d = x - y


    if zero_method == "wilcox":
        # Keep all non-zero differences
        d = compress(np.not_equal(d, 0), d, axis=-1)


    count = len(d)
    if count < 10:
        warnings.warn("Warning: sample size too small for normal approximation.")


    r = stats.rankdata(abs(d))
    r_plus = np.sum((d > 0) * r, axis=0)
    r_minus = np.sum((d < 0) * r, axis=0)


    if zero_method == "zsplit":
        r_zero = np.sum((d == 0) * r, axis=0)
        r_plus += r_zero / 2.
        r_minus += r_zero / 2.


    T = min(r_plus, r_minus)
    mn = count * (count + 1.) * 0.25
    se = count * (count + 1.) * (2. * count + 1.)


    if zero_method == "pratt":
        r = r[d != 0]


    replist, repnum = find_repeats(r)
    if repnum.size != 0:
        # Correction for repeated elements.
        se -= 0.5 * (repnum * (repnum * repnum - 1)).sum()


    se = sqrt(se / 24)
    correction = 0.5 * int(bool(correction)) * np.sign(T - mn)
    z = (T - mn - correction) / se
    prob = 2. * distributions.norm.sf(abs(z))


    return T, prob, z